In [1]:
import keras

In [2]:
keras.__version__

'2.10.0'

In [3]:
import tensorflow

In [4]:
tensorflow.__version__

'2.10.0'

In [5]:
from keras.datasets import mnist

In [6]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import ELU, PReLU, LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers.legacy import Adam
import matplotlib.pyplot as plt
import numpy as np

In [7]:
#Define input image dimensions
#Large images take too much time and resources.
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)


In [8]:
#Given input of noise (latent) vector, the Generator produces an image.
def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)

#Define the generator network 
#Here Dense layers are used. 
    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)    #Generated image

    return Model(noise, img)

In [9]:
#Alpha — α is a hyperparameter which controls the underlying value to which the
#function saturates negatives network inputs.
#Given an input image, the Discriminator outputs the likelihood of the image being real.
    #Binary classification - true or false (we're calling it validity)

def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)
#The validity is the Discriminator’s guess of input being real or not.

In [10]:
#Now that we have constructed our two models it’s time to pit them against each other.
#We do this by defining a training function, loading the data set, re-scaling our training
#images and setting the ground truths. 
def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

#Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
    X_train = np.expand_dims(X_train, axis=3) 

    half_batch = int(batch_size / 2)


#We then loop through a number of epochs to train our Discriminator by first selecting
#a random batch of images from our true dataset, generating a set of images from our
#Generator, feeding both set of images into our Discriminator, and finally setting the
#loss parameters for both the real and fake images, as well as the combined loss. 
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

 
        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of fake images
        gen_imgs = generator.predict(noise)

        # Train the discriminator on real and fake images, separately
        #Research showed that separate training is more effective. 
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    #take average loss from real and fake images. 
    #
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

#And within the same loop we train our Generator, by setting the input noise and
#ultimately training the Generator to have the Discriminator label its samples as valid
#by specifying the gradient loss.
        # ---------------------
        #  Train Generator
        # ---------------------
#Create noise vectors as input for generator. 
#Create as many noise vectors as defined by the batch size. 
#Based on normal distribution. Output will be of size (batch size, 100)
        noise = np.random.normal(0, 1, (batch_size, 100)) 

        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        #This is where the genrator is trying to trick discriminator into believing
        #the generated image is true (hence value of 1 for y)
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

        # Generator is part of combined where it got directly linked with the discriminator
        # Train the generator with noise as x and 1 as y. 
        # Again, 1 as the output as it is adversarial and if generator did a great
        #job of folling the discriminator then the output would be 1 (true)
        g_loss = combined.train_on_batch(noise, valid_y)


#Additionally, in order for us to keep track of our training process, we print the
#progress and save the sample image output depending on the epoch interval specified.  
# Plot the progress
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

#when the specific sample_interval is hit, we call the
#sample_image function. Which looks as follows.


In [11]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view

In [12]:
#Let us also define our optimizer for easy use later on.
#That way if you change your mind, you can change it easily here
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

# Build and compile the discriminator first. 
#Generator will be trained as part of the combined model, later. 
#pick the loss function and the type of metric to keep track.                 
#Binary cross entropy as we are doing prediction and it is a better
#loss function compared to MSE or other. 
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

In [14]:
#build and compile our Discriminator, pick the loss function

#SInce we are only generating (faking) images, let us not track any metrics.
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

##This builds the Generator and defines the input noise. 
#In a GAN the Generator network takes noise z as an input to produce its images.  
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

#This ensures that when we combine our networks we only train the Generator.
#While generator training we do not want discriminator weights to be adjusted. 
#This Doesn't affect the above descriminator training.     
discriminator.trainable = False  

#This specifies that our Discriminator will take the images generated by our Generator
#and true dataset and set its output to a parameter called valid, which will indicate
#whether the input is real or not.  
valid = discriminator(img)  #Validity check on the generated image


#Here we combined the models and also set our loss function and optimizer. 
#Again, we are only training the generator here. 
#The ultimate goal here is for the Generator to fool the Discriminator.  
# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=10000, batch_size=32, save_interval=10)

#Save model for future use to generate fake images
#Not tested yet... make sure right model is being saved..
#Compare with GAN4

generator.save('generator_model.h5')  #Test the model on GAN4_predict...
#Change epochs back to 30K
                
#Epochs dictate the number of backward and forward propagations, the batch_size
#indicates the number of training samples per backward/forward propagation, and the
#sample_interval specifies after how many epochs we call our sample_image function.

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_4 (Batc  (None, 512)              2048      
 hNormalization)                                      

1/1 [==============================] - 0s 12ms/step
54 [D loss: 0.014029, acc.: 100.00%] [G loss: 4.071568]
1/1 [==============================] - 0s 12ms/step
55 [D loss: 0.012909, acc.: 100.00%] [G loss: 4.011705]
1/1 [==============================] - 0s 12ms/step
56 [D loss: 0.023509, acc.: 100.00%] [G loss: 4.000949]
1/1 [==============================] - 0s 13ms/step
57 [D loss: 0.014375, acc.: 100.00%] [G loss: 3.975094]
1/1 [==============================] - 0s 13ms/step
58 [D loss: 0.014763, acc.: 100.00%] [G loss: 4.043396]
1/1 [==============================] - 0s 12ms/step
59 [D loss: 0.017494, acc.: 100.00%] [G loss: 4.043722]
1/1 [==============================] - 0s 12ms/step
60 [D loss: 0.010126, acc.: 100.00%] [G loss: 4.168545]
1/1 [==============================] - 0s 14ms/step
61 [D loss: 0.012326, acc.: 100.00%] [G loss: 4.105717]
1/1 [==============================] - 0s 13ms/step
62 [D loss: 0.013170, acc.: 100.00%] [G loss: 4.219434]
1/1 [=======================

1/1 [==============================] - 0s 11ms/step
126 [D loss: 0.084574, acc.: 96.88%] [G loss: 4.100205]
1/1 [==============================] - 0s 12ms/step
127 [D loss: 0.069458, acc.: 96.88%] [G loss: 4.412559]
1/1 [==============================] - 0s 12ms/step
128 [D loss: 0.088235, acc.: 100.00%] [G loss: 4.089535]
1/1 [==============================] - 0s 12ms/step
129 [D loss: 0.077086, acc.: 96.88%] [G loss: 4.228016]
1/1 [==============================] - 0s 12ms/step
130 [D loss: 0.054960, acc.: 100.00%] [G loss: 3.627862]
1/1 [==============================] - 0s 12ms/step
131 [D loss: 0.207079, acc.: 93.75%] [G loss: 3.722677]
1/1 [==============================] - 0s 16ms/step
132 [D loss: 0.111135, acc.: 93.75%] [G loss: 4.342299]
1/1 [==============================] - 0s 12ms/step
133 [D loss: 0.070489, acc.: 100.00%] [G loss: 4.100219]
1/1 [==============================] - 0s 12ms/step
134 [D loss: 0.146249, acc.: 90.62%] [G loss: 3.897502]
1/1 [====================

1/1 [==============================] - 0s 12ms/step
198 [D loss: 0.421857, acc.: 78.12%] [G loss: 3.088127]
1/1 [==============================] - 0s 13ms/step
199 [D loss: 0.305797, acc.: 84.38%] [G loss: 3.067998]
1/1 [==============================] - 0s 12ms/step
200 [D loss: 0.244203, acc.: 90.62%] [G loss: 3.733432]
1/1 [==============================] - 0s 14ms/step
201 [D loss: 0.173666, acc.: 96.88%] [G loss: 2.697654]
1/1 [==============================] - 0s 12ms/step
202 [D loss: 0.215040, acc.: 93.75%] [G loss: 2.870899]
1/1 [==============================] - 0s 12ms/step
203 [D loss: 0.086521, acc.: 100.00%] [G loss: 3.733355]
1/1 [==============================] - 0s 12ms/step
204 [D loss: 0.769483, acc.: 71.88%] [G loss: 2.090588]
1/1 [==============================] - 0s 12ms/step
205 [D loss: 0.148696, acc.: 96.88%] [G loss: 3.447715]
1/1 [==============================] - 0s 11ms/step
206 [D loss: 0.279115, acc.: 93.75%] [G loss: 3.465204]
1/1 [======================

1/1 [==============================] - 0s 13ms/step
270 [D loss: 0.636066, acc.: 62.50%] [G loss: 1.353246]
1/1 [==============================] - 0s 21ms/step
271 [D loss: 0.563231, acc.: 68.75%] [G loss: 1.468694]
1/1 [==============================] - 0s 12ms/step
272 [D loss: 0.662592, acc.: 56.25%] [G loss: 1.733456]
1/1 [==============================] - 0s 13ms/step
273 [D loss: 0.777346, acc.: 53.12%] [G loss: 0.952609]
1/1 [==============================] - 0s 14ms/step
274 [D loss: 0.668032, acc.: 53.12%] [G loss: 1.346127]
1/1 [==============================] - 0s 12ms/step
275 [D loss: 0.632707, acc.: 59.38%] [G loss: 1.412810]
1/1 [==============================] - 0s 12ms/step
276 [D loss: 1.236100, acc.: 21.88%] [G loss: 0.553689]
1/1 [==============================] - 0s 12ms/step
277 [D loss: 0.532361, acc.: 65.62%] [G loss: 1.600538]
1/1 [==============================] - 0s 13ms/step
278 [D loss: 0.923891, acc.: 43.75%] [G loss: 0.906674]
1/1 [=======================

...

1/1 [==============================] - 0s 12ms/step
9685 [D loss: 0.629664, acc.: 68.75%] [G loss: 0.864805]
1/1 [==============================] - 0s 11ms/step
9686 [D loss: 0.810226, acc.: 37.50%] [G loss: 0.817771]
1/1 [==============================] - 0s 12ms/step
9687 [D loss: 0.592402, acc.: 71.88%] [G loss: 0.891713]
1/1 [==============================] - 0s 11ms/step
9688 [D loss: 0.740586, acc.: 50.00%] [G loss: 0.807797]
1/1 [==============================] - 0s 14ms/step
9689 [D loss: 0.701427, acc.: 43.75%] [G loss: 0.856535]
1/1 [==============================] - 0s 13ms/step
9690 [D loss: 0.615686, acc.: 65.62%] [G loss: 0.916888]
1/1 [==============================] - 0s 12ms/step
9691 [D loss: 0.684843, acc.: 53.12%] [G loss: 0.913221]
1/1 [==============================] - 0s 12ms/step
9692 [D loss: 0.657814, acc.: 53.12%] [G loss: 0.845734]
1/1 [==============================] - 0s 19ms/step
9693 [D loss: 0.666547, acc.: 56.25%] [G loss: 0.860259]
1/1 [==============

1/1 [==============================] - 0s 12ms/step
9757 [D loss: 0.634325, acc.: 71.88%] [G loss: 0.916776]
1/1 [==============================] - 0s 12ms/step
9758 [D loss: 0.700097, acc.: 59.38%] [G loss: 0.856600]
1/1 [==============================] - 0s 12ms/step
9759 [D loss: 0.758130, acc.: 40.62%] [G loss: 0.855419]
1/1 [==============================] - 0s 12ms/step
9760 [D loss: 0.605463, acc.: 68.75%] [G loss: 0.814659]
1/1 [==============================] - 0s 12ms/step
9761 [D loss: 0.700120, acc.: 43.75%] [G loss: 0.908107]
1/1 [==============================] - 0s 12ms/step
9762 [D loss: 0.669841, acc.: 59.38%] [G loss: 0.843917]
1/1 [==============================] - 0s 11ms/step
9763 [D loss: 0.730309, acc.: 53.12%] [G loss: 0.972047]
1/1 [==============================] - 0s 12ms/step
9764 [D loss: 0.725250, acc.: 53.12%] [G loss: 0.919069]
1/1 [==============================] - 0s 11ms/step
9765 [D loss: 0.700245, acc.: 56.25%] [G loss: 0.894116]
1/1 [==============

1/1 [==============================] - 0s 11ms/step
9829 [D loss: 0.660754, acc.: 50.00%] [G loss: 0.851464]
1/1 [==============================] - 0s 11ms/step
9830 [D loss: 0.673566, acc.: 53.12%] [G loss: 0.806862]
1/1 [==============================] - 0s 13ms/step
9831 [D loss: 0.694563, acc.: 53.12%] [G loss: 0.776266]
1/1 [==============================] - 0s 12ms/step
9832 [D loss: 0.643342, acc.: 65.62%] [G loss: 0.796167]
1/1 [==============================] - 0s 12ms/step
9833 [D loss: 0.750287, acc.: 43.75%] [G loss: 0.841009]
1/1 [==============================] - 0s 12ms/step
9834 [D loss: 0.699729, acc.: 53.12%] [G loss: 0.898287]
1/1 [==============================] - 0s 11ms/step
9835 [D loss: 0.714529, acc.: 53.12%] [G loss: 0.885636]
1/1 [==============================] - 0s 12ms/step
9836 [D loss: 0.689638, acc.: 59.38%] [G loss: 0.781768]
1/1 [==============================] - 0s 11ms/step
9837 [D loss: 0.661930, acc.: 62.50%] [G loss: 0.810443]
1/1 [==============

1/1 [==============================] - 0s 38ms/step
9901 [D loss: 0.606321, acc.: 65.62%] [G loss: 0.811843]
1/1 [==============================] - 0s 14ms/step
9902 [D loss: 0.697899, acc.: 43.75%] [G loss: 0.819343]
1/1 [==============================] - 0s 13ms/step
9903 [D loss: 0.705442, acc.: 50.00%] [G loss: 0.840028]
1/1 [==============================] - 0s 13ms/step
9904 [D loss: 0.716522, acc.: 43.75%] [G loss: 0.872683]
1/1 [==============================] - 0s 13ms/step
9905 [D loss: 0.685115, acc.: 56.25%] [G loss: 0.850204]
1/1 [==============================] - 0s 12ms/step
9906 [D loss: 0.693485, acc.: 56.25%] [G loss: 0.827723]
1/1 [==============================] - 0s 12ms/step
9907 [D loss: 0.696396, acc.: 50.00%] [G loss: 0.899605]
1/1 [==============================] - 0s 12ms/step
9908 [D loss: 0.732673, acc.: 46.88%] [G loss: 0.879078]
1/1 [==============================] - 0s 14ms/step
9909 [D loss: 0.707194, acc.: 53.12%] [G loss: 0.919521]
1/1 [==============

1/1 [==============================] - 0s 11ms/step
9972 [D loss: 0.652874, acc.: 59.38%] [G loss: 0.864028]
1/1 [==============================] - 0s 14ms/step
9973 [D loss: 0.663289, acc.: 71.88%] [G loss: 0.871548]
1/1 [==============================] - 0s 12ms/step
9974 [D loss: 0.674394, acc.: 59.38%] [G loss: 0.820637]
1/1 [==============================] - 0s 12ms/step
9975 [D loss: 0.692022, acc.: 46.88%] [G loss: 0.853248]
1/1 [==============================] - 0s 11ms/step
9976 [D loss: 0.795090, acc.: 31.25%] [G loss: 0.855920]
1/1 [==============================] - 0s 12ms/step
9977 [D loss: 0.695471, acc.: 50.00%] [G loss: 0.978528]
1/1 [==============================] - 0s 12ms/step
9978 [D loss: 0.659884, acc.: 62.50%] [G loss: 0.887480]
1/1 [==============================] - 0s 11ms/step
9979 [D loss: 0.726117, acc.: 50.00%] [G loss: 0.873955]
1/1 [==============================] - 0s 12ms/step
9980 [D loss: 0.707099, acc.: 46.88%] [G loss: 0.841777]
1/1 [==============